# レッスン5: MIDIとシーケンサープログラミング

前回まで学んだこと：
- 音響合成の基礎（発振器、エンベロープ、フィルター）
- オーディオエフェクト（リバーブ、ディレイ、コーラス）
- 楽器音の設計

今回のレッスンで学ぶこと：
- **MIDI**システムの基本概念
- **シーケンサー**による音楽データの管理
- **トラック**と**チャンネル**の概念
- **自動演奏**システムの構築
- **リズムパターン**の作成

🎵 ゴール：プログラムで音楽を自動演奏し、複数楽器による簡単なアンサンブルを作成

## 準備：ライブラリのセットアップ

In [ ]:
# 🛠️ 環境判定とライブラリセットアップ
import sys
import os

# Google Colab環境かどうかを判定
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Google Colab環境で実行中...")
except ImportError:
    IN_COLAB = False
    print("🏠 ローカル環境で実行中")

# ライブラリのセットアップ
if IN_COLAB:
    print("🔧 Google Colab環境を設定中...")
    
    # 必要なパッケージをインストール
    !pip install numpy scipy matplotlib ipython japanize-matplotlib
    
    # GitHubからライブラリをクローン
    !git clone https://github.com/ggszk/simple-audio-programming.git
    
    # パスを追加
    sys.path.append('/content/simple-audio-programming')
    
    print("✅ セットアップ完了！")
    print("📝 このノートブックを自分用にコピーするには:")
    print("   ファイル → ドライブにコピーを保存")
    
else:
    # ローカル環境では、パッケージが存在するかチェック
    try:
        import audio_lib
        print("audio_libが既にインストールされています")
    except ImportError:
        # パッケージが見つからない場合、パスを追加
        current_dir = os.path.dirname(os.path.abspath('.'))
        parent_dir = os.path.dirname(current_dir)
        if os.path.exists(os.path.join(parent_dir, 'audio_lib')):
            sys.path.insert(0, parent_dir)
            print(f"パスを追加しました: {parent_dir}")
        else:
            # Poetry環境の場合はパスを直接追加
            project_root = os.path.dirname(current_dir)
            if os.path.exists(os.path.join(project_root, 'audio_lib')):
                sys.path.insert(0, project_root)
                print(f"Poetryプロジェクトのパスを追加しました: {project_root}")
            else:
                print("audio_libディレクトリが見つかりません")

# インポート文（メインモジュールから直接インポート）
from audio_lib import (
    AudioConfig, SineWave, ADSREnvelope, 
    note_to_frequency, frequency_to_note, save_audio
)
from audio_lib.synthesis.envelopes import apply_envelope
from audio_lib.sequencer import AudioSequencer
from audio_lib.instruments.basic_instruments import BasicSynth
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt

# 日本語フォント設定（Colab用）
if IN_COLAB:
    import japanize_matplotlib
    print("✅ 日本語フォントを設定しました")
import warnings
warnings.filterwarnings('ignore')

config = AudioConfig()

print("\n🎵 MIDIとシーケンサーの学習を始めましょう！")

## 5.1 MIDIとは何か？

**MIDI**（Musical Instrument Digital Interface）は、音楽の演奏情報をデジタルで表現する規格です。

### MIDIの基本概念：
- **ノート**：音程と音の長さの情報
- **チャンネル**：楽器の種類を分ける（16チャンネル）
- **ベロシティ**：音の強さ（音量）
- **タイミング**：いつ音を鳴らすか

### 🎹 オーディオとMIDIの違い：
- **オーディオ**：実際の音の波形データ
- **MIDI**：「いつ、どの鍵盤を、どの強さで押したか」の情報

In [ ]:
# MIDIノートの基本概念を理解しよう

# ノートクラスの使い方
print("=== MIDIノートの基本 ===")

# 1つのノート（音符）を作成
note1 = Note(
    note_number=60,  # C4（ド）のMIDIノート番号
    velocity=100,    # 音の強さ（0-127）
    start_time=0.0,  # 開始時刻（秒）
    duration=1.0     # 音の長さ（秒）
)

print(f"ノート情報:")
print(f"  ノート番号: {note1.note_number} (C4)")
print(f"  ベロシティ: {note1.velocity}")
print(f"  開始時刻: {note1.start_time}秒")
print(f"  長さ: {note1.duration}秒")
print(f"  周波数: {note1.get_frequency():.2f}Hz")

# MIDI ノート番号と音名の対応を確認
note_examples = [
    (60, "C4"), (61, "C#4"), (62, "D4"), (63, "D#4"), (64, "E4"),
    (65, "F4"), (66, "F#4"), (67, "G4"), (68, "G#4"), (69, "A4"),
    (70, "A#4"), (71, "B4"), (72, "C5")
]

print("\n=== ノート番号と音名の対応 ===")
print("MIDI番号 | 音名 | 周波数(Hz)")
print("-" * 30)
for note_num, note_name in note_examples:
    freq = note_to_frequency(note_num)
    print(f"ノート番号 {note_num:2d} = {note_name} = {freq:6.2f}Hz")

## 5.2 シーケンサーの基本：トラックとタイムライン

In [ ]:
# シーケンサーでシンプルなメロディーを作成

# 1. トラック（楽器パート）を作成
melody_track = Track(name="Piano Melody")

# 2. ノートを追加（"きらきら星"の最初の部分）
melody_notes = [
    # ド ド ソ ソ ラ ラ ソ
    Note(60, 100, 0.0, 0.5),  # C4
    Note(60, 100, 0.5, 0.5),  # C4
    Note(67, 100, 1.0, 0.5),  # G4
    Note(67, 100, 1.5, 0.5),  # G4
    Note(69, 100, 2.0, 0.5),  # A4
    Note(69, 100, 2.5, 0.5),  # A4
    Note(67, 100, 3.0, 1.0),  # G4（長め）
]

# トラックにノートを追加
for note in melody_notes:
    melody_track.add_note(note)

print(f"トラック '{melody_track.name}' に {len(melody_track.notes)} 個のノートを追加")

# 3. シーケンサーにトラックを登録
sequencer = Sequencer(config)
sequencer.add_track(melody_track)

# 4. 楽器（音源）を設定
piano = BasicPiano()
sequencer.set_instrument(melody_track.name, piano)

print("シーケンサーの設定完了！")

In [ ]:
# シーケンサーで音楽を演奏
print("きらきら星（最初の部分）をシーケンサーで演奏:")

# 演奏してオーディオデータを生成
performance_audio = sequencer.render(duration=4.0)

# 再生
display(Audio(performance_audio, rate=config.sample_rate))

# タイムラインの可視化
plt.figure(figsize=(12, 4))
for i, note in enumerate(melody_notes):
    start = note.start_time
    end = start + note.duration
    height = note.note_number
    
    plt.barh(height, note.duration, left=start, height=1, 
             alpha=0.7, color=f'C{i%10}', 
             label=f'Note {note.note_number}')
    
    # ノート名を表示
    note_name = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"][note.note_number % 12]
    octave = note.note_number // 12 - 1
    plt.text(start + note.duration/2, height, f'{note_name}{octave}', 
             ha='center', va='center', fontweight='bold')

plt.xlabel('時間 (秒)')
plt.ylabel('MIDIノート番号')
plt.title('メロディーのタイムライン')
plt.grid(True, alpha=0.3)
plt.ylim(58, 72)
plt.xlim(0, 4)
plt.show()

## 5.3 複数トラック：アンサンブル演奏

In [ ]:
# 複数の楽器によるアンサンブルを作成

# 新しいシーケンサーを作成
ensemble_sequencer = Sequencer(config)

# === トラック1: メロディー（ピアノ） ===
melody_track = Track(name="Piano")
melody_notes = [
    # C-E-G-C（アルペジオ）
    Note(60, 100, 0.0, 0.5),  # C4
    Note(64, 100, 0.5, 0.5),  # E4
    Note(67, 100, 1.0, 0.5),  # G4
    Note(72, 100, 1.5, 0.5),  # C5
    
    Note(72, 100, 2.0, 0.5),  # C5
    Note(67, 100, 2.5, 0.5),  # G4
    Note(64, 100, 3.0, 0.5),  # E4
    Note(60, 100, 3.5, 0.5),  # C4
]

for note in melody_notes:
    melody_track.add_note(note)

# === トラック2: ベース（オルガン低音） ===
bass_track = Track(name="Bass")
bass_notes = [
    Note(36, 120, 0.0, 2.0),  # C2（低いド）
    Note(36, 120, 2.0, 2.0),  # C2
]

for note in bass_notes:
    bass_track.add_note(note)

# === トラック3: ドラム（リズム） ===
drum_track = Track(name="Drums")
# キック（低音）とハイハット（高音）でシンプルなビート
drum_notes = [
    # キック（ノート番号36 = C2をキックドラムとして使用）
    Note(36, 127, 0.0, 0.1),
    Note(36, 127, 1.0, 0.1),
    Note(36, 127, 2.0, 0.1),
    Note(36, 127, 3.0, 0.1),
    
    # ハイハット（ノート番号42をハイハットとして使用）
    Note(42, 80, 0.5, 0.1),
    Note(42, 80, 1.5, 0.1),
    Note(42, 80, 2.5, 0.1),
    Note(42, 80, 3.5, 0.1),
]

for note in drum_notes:
    drum_track.add_note(note)

# シーケンサーにトラックを追加
ensemble_sequencer.add_track(melody_track)
ensemble_sequencer.add_track(bass_track)
ensemble_sequencer.add_track(drum_track)

# 楽器を設定
piano = BasicPiano()
organ = BasicOrgan()
drums = BasicDrum()

ensemble_sequencer.set_instrument("Piano", piano)
ensemble_sequencer.set_instrument("Bass", organ)
ensemble_sequencer.set_instrument("Drums", drums)

print("3つのトラックによるアンサンブルを設定完了！")
print(f"トラック数: {len(ensemble_sequencer.tracks)}")

In [ ]:
# アンサンブル演奏
print("アンサンブル演奏（ピアノ + ベース + ドラム）:")

ensemble_audio = ensemble_sequencer.render(duration=4.0)
display(Audio(ensemble_audio, rate=config.sample_rate))

# 各トラックを個別に聞いてみよう
print("\n=== 各トラック個別再生 ===")

# ピアノのみ
piano_only = Sequencer(config)
piano_only.add_track(melody_track)
piano_only.set_instrument("Piano", piano)
piano_audio = piano_only.render(duration=4.0)

print("ピアノのみ:")
display(Audio(piano_audio, rate=config.sample_rate))

# ベースのみ
bass_only = Sequencer(config)
bass_only.add_track(bass_track)
bass_only.set_instrument("Bass", organ)
bass_audio = bass_only.render(duration=4.0)

print("ベースのみ:")
display(Audio(bass_audio, rate=config.sample_rate))

# ドラムのみ
drum_only = Sequencer(config)
drum_only.add_track(drum_track)
drum_only.set_instrument("Drums", drums)
drum_audio = drum_only.render(duration=4.0)

print("ドラムのみ:")
display(Audio(drum_audio, rate=config.sample_rate))

## 5.4 リズムパターンの作成

In [ ]:
# 様々なリズムパターンを作成する関数

def create_drum_pattern(pattern_name="basic", duration=4.0, tempo=120):
    """ドラムパターンを作成
    
    Args:
        pattern_name: パターンの種類
        duration: パターンの長さ（秒）
        tempo: テンポ（BPM）
    """
    track = Track(name=f"Drums_{pattern_name}")
    beat_length = 60.0 / tempo  # 1拍の長さ（秒）
    
    if pattern_name == "basic":
        # 基本的な4/4ビート
        for beat in range(int(duration / beat_length)):
            time = beat * beat_length
            
            # キック（1拍目と3拍目）
            if beat % 4 in [0, 2]:
                track.add_note(Note(36, 127, time, 0.1))
            
            # スネア（2拍目と4拍目）
            if beat % 4 in [1, 3]:
                track.add_note(Note(38, 100, time, 0.1))
            
            # ハイハット（全拍）
            track.add_note(Note(42, 60, time, 0.05))
    
    elif pattern_name == "rock":
        # ロック調のビート
        for beat in range(int(duration / (beat_length/2))):
            time = beat * (beat_length/2)
            
            # キック（強拍）
            if beat % 8 in [0, 3, 6]:
                track.add_note(Note(36, 127, time, 0.1))
            
            # スネア（裏拍強調）
            if beat % 8 in [2, 6]:
                track.add_note(Note(38, 110, time, 0.1))
            
            # ハイハット（8分音符）
            track.add_note(Note(42, 70, time, 0.05))
    
    elif pattern_name == "swing":
        # スイング調
        for beat in range(int(duration / (beat_length/3))):
            time = beat * (beat_length/3)
            
            # キック
            if beat % 12 in [0, 6]:
                track.add_note(Note(36, 120, time, 0.1))
            
            # スネア（シャッフル）
            if beat % 12 in [4, 10]:
                track.add_note(Note(38, 90, time, 0.1))
            
            # ライド（三連符）
            if beat % 3 == 0:
                track.add_note(Note(51, 50, time, 0.1))
    
    return track

# 異なるリズムパターンを試してみよう
patterns = ["basic", "rock", "swing"]

for pattern in patterns:
    drum_track = create_drum_pattern(pattern, duration=4.0, tempo=120)
    
    pattern_sequencer = Sequencer(config)
    pattern_sequencer.add_track(drum_track)
    pattern_sequencer.set_instrument(drum_track.name, BasicDrum())
    
    pattern_audio = pattern_sequencer.render(duration=4.0)
    
    print(f"{pattern.upper()}ビート:")
    display(Audio(pattern_audio, rate=config.sample_rate))
    print()

## 5.5 コード進行の自動生成

In [ ]:
# コード進行を自動生成する関数

def create_chord_progression(root_note=60, progression="I-V-vi-IV", duration_per_chord=2.0):
    """コード進行を作成
    
    Args:
        root_note: ルート音のMIDIノート番号
        progression: コード進行のパターン
        duration_per_chord: 1つのコードの長さ（秒）
    """
    track = Track(name="Chords")
    
    # コードの構成音（メジャースケール）
    scale = [0, 2, 4, 5, 7, 9, 11]  # C major scale intervals
    
    # コード進行の定義
    if progression == "I-V-vi-IV":
        # よくあるポップスの進行
        chord_sequence = [
            [0, 2, 4],    # I (C-E-G)
            [4, 6, 1],    # V (G-B-D)
            [5, 0, 2],    # vi (A-C-E)
            [3, 5, 0],    # IV (F-A-C)
        ]
    elif progression == "vi-IV-I-V":
        # 「カノン進行」
        chord_sequence = [
            [5, 0, 2],    # vi (A-C-E)
            [3, 5, 0],    # IV (F-A-C)
            [0, 2, 4],    # I (C-E-G)
            [4, 6, 1],    # V (G-B-D)
        ]
    else:
        # デフォルト：簡単な進行
        chord_sequence = [
            [0, 2, 4],    # I
            [3, 5, 0],    # IV
            [4, 6, 1],    # V
            [0, 2, 4],    # I
        ]
    
    # コードを生成
    for chord_idx, chord_intervals in enumerate(chord_sequence):
        start_time = chord_idx * duration_per_chord
        
        for interval in chord_intervals:
            note_number = root_note + scale[interval]
            note = Note(
                note_number=note_number,
                velocity=80,
                start_time=start_time,
                duration=duration_per_chord * 0.9  # 少し短めで重なりを防ぐ
            )
            track.add_note(note)
    
    return track

# 異なるコード進行を試してみよう
progressions = ["I-V-vi-IV", "vi-IV-I-V"]

for prog in progressions:
    chord_track = create_chord_progression(root_note=48, progression=prog, duration_per_chord=1.5)
    
    chord_sequencer = Sequencer(config)
    chord_sequencer.add_track(chord_track)
    chord_sequencer.set_instrument("Chords", BasicOrgan())
    
    chord_audio = chord_sequencer.render(duration=6.0)
    
    print(f"{prog}進行:")
    display(Audio(chord_audio, rate=config.sample_rate))
    print()

## 5.6 実践演習：フルアレンジメントの作成

In [ ]:
# 学習した技術を組み合わせて本格的な楽曲を作成

def create_full_arrangement():
    """フルアレンジメントの作成"""
    
    full_sequencer = Sequencer(config)
    
    # === 1. ドラムトラック ===
    drum_track = create_drum_pattern("rock", duration=8.0, tempo=120)
    full_sequencer.add_track(drum_track)
    full_sequencer.set_instrument(drum_track.name, BasicDrum())
    
    # === 2. ベースライン ===
    bass_track = Track(name="Bass")
    # シンプルなベースライン（ルート音中心）
    bass_notes = [
        # C - G - Am - F の進行に合わせたベース
        Note(36, 120, 0.0, 1.5),   # C
        Note(36, 100, 1.5, 0.5),  # C
        Note(43, 120, 2.0, 1.5),  # G
        Note(43, 100, 3.5, 0.5),  # G
        Note(45, 120, 4.0, 1.5),  # A
        Note(45, 100, 5.5, 0.5),  # A
        Note(41, 120, 6.0, 1.5),  # F
        Note(41, 100, 7.5, 0.5),  # F
    ]
    
    for note in bass_notes:
        bass_track.add_note(note)
    
    full_sequencer.add_track(bass_track)
    full_sequencer.set_instrument("Bass", BasicOrgan())
    
    # === 3. コード進行 ===
    chord_track = create_chord_progression(root_note=60, progression="I-V-vi-IV", duration_per_chord=2.0)
    full_sequencer.add_track(chord_track)
    full_sequencer.set_instrument("Chords", BasicPiano())
    
    # === 4. メロディー ===
    melody_track = Track(name="Melody")
    # C major scaleを使った簡単なメロディー
    melody_notes = [
        Note(72, 100, 0.0, 0.5),   # C5
        Note(74, 100, 0.5, 0.5),   # D5
        Note(76, 100, 1.0, 1.0),   # E5
        Note(74, 100, 2.0, 0.5),   # D5
        Note(72, 100, 2.5, 0.5),   # C5
        Note(79, 100, 3.0, 1.0),   # G5
        Note(77, 100, 4.0, 1.0),   # F5
        Note(76, 100, 5.0, 1.0),   # E5
        Note(74, 100, 6.0, 1.0),   # D5
        Note(72, 110, 7.0, 1.0),   # C5
    ]
    
    for note in melody_notes:
        melody_track.add_note(note)
    
    full_sequencer.add_track(melody_track)
    full_sequencer.set_instrument("Melody", BasicPiano())
    
    return full_sequencer

# フルアレンジメントを作成・演奏
print("=== フルアレンジメント（4トラック） ===")
print("ドラム + ベース + コード + メロディー")

full_arrangement = create_full_arrangement()
full_audio = full_arrangement.render(duration=8.0)

display(Audio(full_audio, rate=config.sample_rate))

print(f"\n使用トラック数: {len(full_arrangement.tracks)}")
for track_name in full_arrangement.tracks.keys():
    track = full_arrangement.tracks[track_name]
    print(f"  - {track_name}: {len(track.notes)}個のノート")

## 5.7 チャレンジ課題

### 🎯 課題1：自動作曲アルゴリズム

In [ ]:
import random

def generate_random_melody(scale_notes, num_notes=8, duration_per_note=0.5):
    """ランダムなメロディーを生成
    
    Args:
        scale_notes: 使用するスケールの音程リスト
        num_notes: 生成する音符の数
        duration_per_note: 1音符の長さ
    """
    track = Track(name="Random_Melody")
    
    for i in range(num_notes):
        # ランダムに音程を選択
        note_number = random.choice(scale_notes)
        
        # ランダムにベロシティを選択（80-120の範囲）
        velocity = random.randint(80, 120)
        
        # ランダムに音の長さを選択
        duration = random.choice([0.25, 0.5, 0.75, 1.0]) * duration_per_note
        
        start_time = i * duration_per_note
        
        note = Note(note_number, velocity, start_time, duration)
        track.add_note(note)
    
    return track

# C major scale
c_major_scale = [60, 62, 64, 65, 67, 69, 71, 72]  # C4 to C5

# ランダムメロディーを3つ生成
print("=== 自動作曲：ランダムメロディー ===")

for i in range(3):
    random_melody = generate_random_melody(c_major_scale, num_notes=8, duration_per_note=0.5)
    
    melody_sequencer = Sequencer(config)
    melody_sequencer.add_track(random_melody)
    melody_sequencer.set_instrument("Random_Melody", BasicPiano())
    
    melody_audio = melody_sequencer.render(duration=4.0)
    
    print(f"ランダムメロディー {i+1}:")
    display(Audio(melody_audio, rate=config.sample_rate))
    print()

### 🎯 課題2：テンポ変化とダイナミクス

In [ ]:
def create_dynamic_performance():
    """ダイナミクスとテンポが変化する演奏"""
    
    track = Track(name="Dynamic")
    
    # スケール
    scale = [60, 62, 64, 65, 67, 69, 71, 72]  # C major
    
    # セクション1：静かで遅い（pp, adagio）
    current_time = 0.0
    for i in range(4):
        note_number = scale[i % len(scale)]
        velocity = 40 + i * 5  # 徐々に大きく
        duration = 1.0  # ゆっくり
        
        track.add_note(Note(note_number, velocity, current_time, duration))
        current_time += duration
    
    # セクション2：中間（mp, moderato）
    for i in range(4):
        note_number = scale[(i+2) % len(scale)]
        velocity = 70 + i * 5
        duration = 0.75  # 少し速く
        
        track.add_note(Note(note_number, velocity, current_time, duration))
        current_time += duration
    
    # セクション3：速くて大きく（ff, allegro）
    for i in range(8):
        note_number = scale[(i+4) % len(scale)]
        velocity = 100 + i * 3
        duration = 0.25  # 速く
        
        track.add_note(Note(note_number, velocity, current_time, duration))
        current_time += duration
    
    # セクション4：フィナーレ（fff）
    for i in range(4):
        note_number = scale[7-i]  # 下降
        velocity = 127  # 最大音量
        duration = 0.5
        
        track.add_note(Note(note_number, velocity, current_time, duration))
        current_time += duration
    
    return track

# ダイナミック演奏を作成
dynamic_track = create_dynamic_performance()

dynamic_sequencer = Sequencer(config)
dynamic_sequencer.add_track(dynamic_track)
dynamic_sequencer.set_instrument("Dynamic", BasicPiano())

dynamic_audio = dynamic_sequencer.render(duration=10.0)

print("ダイナミクス変化の演奏:")
print("静かで遅く → 中間 → 速くて大きく → フィナーレ")
display(Audio(dynamic_audio, rate=config.sample_rate))

# ベロシティの変化を可視化
plt.figure(figsize=(12, 6))
times = [note.start_time for note in dynamic_track.notes]
velocities = [note.velocity for note in dynamic_track.notes]

plt.subplot(2, 1, 1)
plt.plot(times, velocities, 'o-', linewidth=2, markersize=6)
plt.ylabel('ベロシティ')
plt.title('ダイナミクス変化')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
note_numbers = [note.note_number for note in dynamic_track.notes]
plt.plot(times, note_numbers, 's-', linewidth=2, markersize=6, color='orange')
plt.xlabel('時間 (秒)')
plt.ylabel('音程')
plt.title('メロディーライン')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## まとめ

### 今回学んだこと：
1. **MIDI**システムの基本概念（ノート、ベロシティ、タイミング）
2. **シーケンサー**による音楽データの管理
3. **トラック**と**楽器**の組み合わせ
4. **リズムパターン**の作成技法
5. **コード進行**の自動生成
6. **アンサンブル**演奏の構築
7. **自動作曲**の基礎

### 🎵 音楽プログラミングのコツ：
- **レイヤー思考**：各楽器パートを独立して考える
- **タイミングの重要性**：音楽はリズムが命
- **音量バランス**：各楽器の音量関係を調整
- **音色の住み分け**：各楽器が干渉しないよう配慮

### 次回予告：レッスン6
- **サンプリング**と**音声分析**
- **既存音楽の解析**
- **音声認識**と**ピッチ検出**
- **リアルタイム処理**の基礎

### 🏠 宿題
1. 自分の好きな楽曲のコード進行を調べて、シーケンサーで再現してみよう
2. 3つ以上の楽器によるオリジナルアンサンブルを作成してみよう
3. ランダム要素を含む自動作曲アルゴリズムを改良してみよう